In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf
import transformers
import time


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Проведем тесты на 3 моделях, которые могут работать с мультиязыковыми моделями:

1) Берт , обученный на 100 языках

2)XLM

3) XLM-Roberta

BERT

Это трансформер , состоящий из некоторого количества Encoder слоев в зависимости от конфигурации(12 или 24). Каждый энкодер слой состоит из настаканных multi-head attention с Dropout, LayerNorm и FeedForward слоями внутри.

Берт обучается на следующих задачах:

1) Предсказание замаскированных слов в документе на основе остальных (MLM)

2) Предсказание одного предложения в документе на основе предыдущего (NSP)

На вход в модель данные должны быть конвертированы в индексы из словаря токенайзера для конкретной конфигурации модели, после чего по каждому индексу достается эмбединг токена, который поэлементно суммируется с векторными представлениями позиции токена в предложении и токена места предложения в документе.

На выход Берт для одного документа возвращает матрицу размером max_len x embed_size, где по строчкам эмбединги каждого из токенов документа. В первом ряду этой матрицы находится эмбединг так называемого cls токена, который авторы модели советуют использовать как эмбединг всего документа, хотя это и не единственный возможный способ и не всегда самый эффективный.


XLM

Это модель тоже основанная архитектуре берта. Только эта модель отличается тем, что она лучше адаптирована для межязыковых задач. Это возможно засчет отчасти других подходов к обучению. А именно модель обучалась с задачами:

1) Предсказание слова в предложении на основе всех предыдущих (CLM)

2) Предсказание замаскированных слов предложения на основе всех остальных (MLM)

3) Предсказание замаскированных слов на основе предыдущих, только за тем исключением, что входные данные представляются из себя конкатенацию одного и того предложения но на двух языках (TLM)

Эти методы обучения были взяты за основу в следующей модели - забустченной XLM модели на основе берта - XLM-Roberta


XLM-Roberta 

Эта модель повторяет архитектуру Берта, но как пишут ее создатели , она превосходит сильно результаты берта на межязыковых задачах (спойлер это реально так). Лучшее качество удалось получить за счет измененного процесса обучения, а именно: модель обучалась на следующих задачах:

1) Предсказание слова в предложении на основе всех предыдущих (CLM)

2) Предсказание замаскированных слов предложения на основе всех остальных (MLM)

3) Предсказание замаскированных слов на основе предыдущих, только за тем исключением, что входные данные представляются из себя конкатенацию одного и того предложения но на двух языках (TLM)

Но кроме этого для обучения было собрано 2.5Т данных, и модель обучалась на большем learning rate, batch size, и с большим количеством эпох - как это делалось в Roberta - дообученной версии берта. Таким образом XLM-R это дообученный берт, который обучался на межязыковых данных и с задачами способствующими межязыковому понимаю языка.


In [ ]:
bert_checkpoint = "bert-base-multilingual-cased" #12-layer, 768-hidden, 12-heads, 110M parameters.
xlm_checkpoint = "xlm-mlm-100-1280"
xlm_roberta_checkpoint = "xlm-roberta-base" #~125M parameters with 12-layers, 768-hidden-state, 3072 feed-forward hidden-state, 8-heads


Возьмем часть данных - датасет с первого соревнования Jigsaw, заенкодим их к длине в 128 токенов и запустим 3 модели на одной эпохе, различающихся только трансформером, создающем ембединги входящих текстовых данных, чтобы посмотреть какая из моделей даст лучшее качество на валидационной выборке при прочих равных.

In [ ]:
max_len = 128

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained(bert_checkpoint)
xlm_tokenizer = AutoTokenizer.from_pretrained(xlm_checkpoint)
xlm_roberta_tokenizer = AutoTokenizer.from_pretrained(xlm_roberta_checkpoint)

In [ ]:
#создадим функцию токенизации датасета
def encode_comments(dataframe, tokenizer, max_len=max_len):
    pos = 0
    start = time.time()
    
    while pos < len(dataframe):
        temp = dataframe[pos:pos+10000].copy()
        res = tokenizer.batch_encode_plus(temp.comment_text.values,
                                          pad_to_max_length=True,
                                          max_length = max_len,
                                          return_attention_masks = False
                                         )
        if pos == 0:
            ids = np.array(res["input_ids"])
            labels = temp.toxic.values
        else:
            ids = np.concatenate((ids, np.array(res["input_ids"])))
            labels = np.concatenate((labels, temp.toxic.values))
        pos+=10000
        print("Processed", pos, "elements")
    return ids, labels

In [ ]:
def make_model(transformer):
    
    inp = tf.keras.layers.Input(shape=(max_len,), dtype="int32")
    X = transformer(inp)[0]
    cls_token = X[:,0,:]
    X = tf.keras.layers.Dropout(0.3)(cls_token)
    X = tf.keras.layers.Dense(1, activation="sigmoid")(X)
    model = tf.keras.Model(inputs=inp, outputs=X)
    return model
    

In [ ]:
train_path = "../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv"
val_path = "../input/jigsaw-multilingual-toxic-comment-classification/validation.csv"
train = pd.read_csv(train_path, usecols=["comment_text", "toxic"])
val = pd.read_csv(val_path, usecols=["comment_text", "toxic"])

In [ ]:
ids_1, labels_1 = encode_comments(train, tokenizer=bert_tokenizer)
ids_2, labels_2 = encode_comments(train, tokenizer=xlm_tokenizer)
ids_3, labels_3 = encode_comments(train, tokenizer=xlm_roberta_tokenizer)

val_ids_1, val_labels_1 = encode_comments(val, tokenizer=bert_tokenizer)
val_ids_2, val_labels_2 = encode_comments(val, tokenizer=xlm_tokenizer)
val_ids_3, val_labels_3 = encode_comments(val, tokenizer=xlm_roberta_tokenizer)


In [ ]:
len(val)

In [ ]:
encode_comments(val, tokenizer=bert_tokenizer)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
print(tpu_strategy.num_replicas_in_sync)

In [ ]:
batch_size = 16 * tpu_strategy.num_replicas_in_sync


In [ ]:
with tpu_strategy.scope():
    bert = transformers.TFBertModel.from_pretrained(bert_checkpoint)
    m1 = make_model(bert)
    m1.compile(optimizer=tf.keras.optimizers.Adam(3e-5), loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()])
    m1.summary()

In [ ]:
h1 = m1.fit(ids_1, labels_1, batch_size=batch_size, validation_data=(val_ids_1, val_labels_1), epochs=1)

In [ ]:
del m1

In [ ]:
with tpu_strategy.scope():
    xlm_r = transformers.TFXLMRobertaModel.from_pretrained("jplu/tf-xlm-roberta-base")
    m3 = make_model(xlm_r)
    m3.compile(optimizer=tf.keras.optimizers.Adam(3e-5), loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()])
    m3.summary()

In [ ]:
with tpu_strategy.scope():
    xlm = transformers.TFXLMModel.from_pretrained(xlm_checkpoint)
    m2 = make_model(xlm)
    m2.compile(optimizer=tf.keras.optimizers.Adam(3e-5), loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()])
    m2.summary()

In [ ]:
h3 = m3.fit(ids_3, labels_3, batch_size=batch_size, validation_data=(val_ids_3, val_labels_3), epochs=1)

In [ ]:
del m3

In [ ]:
h2 = m2.fit(ids_2, labels_2, batch_size=batch_size, validation_data=(val_ids_2, val_labels_2), epochs=1)

In [ ]:
del m2